# **DeepLandforms - v2**

Author: giacomo.nodjoumi@hyranet.info - g.nodjoumi@jacobs-university.de

## DeepLandforms

With this notebook, users can use train [YOLOv8](https://github.com/ultralytics/ultralytics) models for object detection and instance segmentation models on custom dataset of georeferenced images.
Results can be visualized directly in the noteboo using leafmap and WMS backend.

This notebook includes:
* customizable augmentations using albumentation package
* data train/valid split

## Usage

* Put or link the dataset into the **DeepLandforms** *.env* file
* Run docker-compose up
* Edit the *configs* section by editing the following parameters:

## Parameters
 ------------------------------------------------------------------
| **Parameter** | **Description** | **Example** |
| ---- | ---- | ---- |
| **data_dir** | local path of the data dir |  | /home/user/data |
| **device** | device where to run the model | cuda or cpu |
------------------------------------------------------------------
Then just execute the notebook and monitor the training in **Tensorboard** container.

## Funding
*This study is within the Europlanet 2024 RI and EXPLORE project, and it has received funding from the European Union’s Horizon 2020 research and innovation programme under grant agreement No 871149 and No 101004214.*

In [1]:
import albumentations as A
import cv2 as cv
import os
import shutil
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from utils.DataUtils import get_paths
import pandas as pd

In [2]:
basepath = 'extracted_640_1/src/'
home_dir = '/home/Giacomo/data/'
src_path = f'{home_dir}/{basepath}'
image_path = f'{home_dir}{basepath}'
base_dir = src_path +'/train'
train_path = f'{image_path}/train'
#train_dir = f'{image_path}/train'
valid_path = f'{image_path}/val'
test_path = f'{image_path}/test'


In [3]:
if os.path.isdir(base_dir):
    shutil.rmtree(base_dir)
os.makedirs(base_dir)
if os.path.isdir(train_path):
    shutil.rmtree(train_path)

if os.path.isdir(valid_path):
    shutil.rmtree(valid_path)

if os.path.isdir(test_path):
    shutil.rmtree(test_path)

In [4]:
image_list = [f"{image_path}{image}" for image in get_paths(image_path,'tiff')]
label_list = [f"{image_path}{label}" for label in get_paths(image_path,'txt')]
label_list=[label for label in label_list if "classes.txt" not in label]
len(image_list)

484

In [5]:
train_perc=0.7
valid_perc=0.2
test_perc=0.1
train_set, valid_set = train_test_split(image_list, test_size=valid_perc+test_perc, random_state=1,shuffle=False)
valid_set, test_set = train_test_split(valid_set, test_size=test_perc, random_state=1,shuffle=False)

In [6]:
len(train_set)

338

In [7]:
len(valid_set)

131

In [8]:
len(test_set)

15

###### TRAIN DF
tr_df=pd.DataFrame()
for t in train_set:
    tr_df=pd.concat([tr_df,pd.read_csv(t[1], delimiter=' ',header=None)])
number_list = tr_df.reset_index(drop=True)[0]
type_mapping = {0: 'Type-1', 1: 'Type-2', 2: 'Type-3', 3: 'Type-4', 4: 'Crater', 5: 'Crater'}
tr_df['Class'] = [type_mapping[num] for num in tr_df.reset_index(drop=True)[0]]
##### VALID DF
tv_df=pd.DataFrame()
for t in valid_set:
    tv_df=pd.concat([tv_df,pd.read_csv(t[1], delimiter=' ', header=None)])
number_list = tv_df.reset_index(drop=True)[0]
type_mapping = {0: 'Type-1', 1: 'Type-2', 2: 'Type-3', 3: 'Type-4', 4: 'Crater', 5: 'Crater'}
tv_df['Class'] = [type_mapping[num] for num in tv_df.reset_index(drop=True)[0]]
##### TEST DF
tt_df=pd.DataFrame()
for t in test_set:
    tt_df=pd.concat([tt_df,pd.read_csv(t[1], delimiter=' ', header=None)])
number_list = tt_df.reset_index(drop=True)[0]
type_mapping = {0: 'Type-1', 1: 'Type-2', 2: 'Type-3', 3: 'Type-4', 4: 'Crater', 5: 'Crater'}
tt_df['Class'] = [type_mapping[num] for num in tt_df.reset_index(drop=True)[0]]    
dfs = [tr_df, tv_df, tt_df]
dfs = {'Train':tr_df,
      'Valid':tv_df,
      'Test':tt_df}
import matplotlib.pyplot as plt
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

# Iterate over DataFrames and plot pie charts
for i,k in enumerate(dfs):
    
    class_counts = dfs[k]['Class'].value_counts()
    labels = class_counts.index
    sizes = class_counts.values

    axes[i].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    axes[i].axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    axes[i].set_title(f'Distribution in {k}, {dfs[k].shape[0]}')

plt.show()

In [9]:
from pathlib import Path

In [10]:
def dataMoverYOLO(dst_root, image_list):
    try:
        os.makedirs(dst_root)
    except:
        shutil.rmtree(dst_root)
        os.makedirs(dst_root)
    images_path = f"{dst_root}/images"
    os.makedirs(images_path)
    labels_path = f"{dst_root}/labels"        
    os.makedirs(labels_path)
    invalid = []
    for i, image in enumerate(image_list):  
        pathname, ext =os.path.splitext(image)
        label= f"{pathname}.txt"
        name = f"{os.path.basename(pathname)}"
        if os.path.isfile(image) and os.path.isfile(label):
            shutil.copy(image, f"{images_path}/{name}{ext}")
            shutil.copy(label, f"{labels_path}/{name}.txt")
        else:
            invalid.append((image,label))
            
            #print(len(invalid))
    print("Done")
    return(invalid)

In [11]:
dataMoverYOLO(train_path, train_set)

Done


[]

In [12]:
dataMoverYOLO(test_path, test_set)

Done


[]

In [13]:
dataMoverYOLO(valid_path, valid_set)

Done


[]


bg_list = [f"{bg_path}{image}" for image in get_paths(bg_path,'tiff')]
len(bg_list)

for bg in bg_list:
    basename = os.path.basename(bg)
    img_to_path = f'{train_path}/images/'
    shutil.copy(bg, f'{img_to_path}/{basename}')   

In [14]:
import torch
import numpy as np
from ultralytics import YOLO
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [15]:
basepath = 'extracted_640_1/src/'
basename = 'extracted_640_1'
home_dir = '/home/Giacomo/data/'
src_path = f'{home_dir}/{basepath}{basename}/'
image_path = f'{home_dir}{basepath}{basename}/'
base_dir = src_path +'/train'
train_path = f'{image_path}/train'
#train_dir = f'{image_path}/train'
valid_path = f'{image_path}/val'
test_path = f'{image_path}/test'


import neptune

run = neptune.init_run(project='hyradus/Test',api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkN2MyNTE1NS03YjBlLTRiMjctOThiZS1lYTQ0YjFlNTNiY2UifQ==")

yaml_file = f"{home_dir}/{basepath}/data.yaml"
bb = 16
imsize=640
version='n'
model = YOLO(f'yolov8{version}.pt')
results = model.train(
    data=yaml_file,
    #imgsz=imsize,
    epochs=100,
    batch=bb,        
    amp=True,
    half=True,
    #optimizer='AdamW',
    #lr0=0.03,    
    #lrf=0.0000005,
    #cos_lr=True,    
    #dropout=0.9,
    val=True,    
    name=f'yolov8{version}_{imsize}_{bb}_{os.path.dirname(basepath)}_27122023_',
    seed=42,
    cache=True,
    augment=True,
    #degrees= 45.0,  # image rotation (+/- deg)
    #translate=0.25,  # image translation (+/- fraction)
    #scale=0.1,  # image scale (+/- gain)
    #shear=0.1,  # image shear (+/- deg)
    #perspective=0.0,  # image perspective (+/- fraction), range 0-0.001
    #flipud=0.5,  # image flip up-down (probability)
    #fliplr=0.5,  # image flip left-right (probability)
    ##mosaic=1,  # image mosaic (probability)
    #mixup=1,  # image mixup (probability)
    #copy_paste=1,  # segment copy-paste (probability)
    #freeze=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    #patience=100,


deterministic=True)

In [16]:
basepath

'extracted_640_1/src/'

In [17]:
epochs=100

yaml_file = '/home/Giacomo/data/robopit/data.yaml'
bb = 16
imsize=640
version='n'
model = YOLO(f'yolov8{version}.pt')
#model = YOLO('/home/Giacomo/DeepLandforms/YOLO/kfold_pit_batch_4_epochs_60_bg_s_/train2/weights/best.pt')
model.train(seed=42, data=yaml_file,epochs=epochs, batch=bb, name='pit_', half=True)#, augment=True, scale=0.25, flipud=0.5, fliplr=0.5, mosaic=0.5, mixup=0.5, copy_paste=0.5)  # include any train arguments
    

New https://pypi.org/project/ultralytics/8.0.231 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11167MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/Giacomo/data/robopit/data.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=pit_2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, 

train: Scanning /home/Giacomo/data/robopit/train/labels.cache... 1017 images, 57 backgrounds, 0 corrupt: 100%|██████████| 1017/1017 [00:00<?, ?it/s]

train: WARNING ⚠️ /home/Giacomo/data/robopit/train/images/PSP_003515_2155_RED_resized_10-0m_H_1_V_0_png.rf.8cbec4cae5b944d342b1b5b849279e26.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/Giacomo/data/robopit/train/images/PSP_005414_1735_RED_resized_10-0m_H_2_V_0_png.rf.3923aee33314e10f769a30002c3644c1.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/Giacomo/data/robopit/train/images/PSP_005414_1735_RED_resized_10-0m_H_2_V_0_png.rf.98e714b7b10a8e5ec24d280a0c4ef436.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /home/Giacomo/data/robopit/train/images/PSP_008037_1610_RED_resized_10-0m_H_4_V_1_png.rf.0b524733930062be4534e5138a9b8da9.jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /home/Giacomo/data/robopit/valid/labels.cache... 97 images, 0 backgrounds, 0 corrupt: 100%|██████████| 97/97 [00:00<?, ?it/s]


Plotting labels to runs/detect/pit_2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/pit_2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         97        252      0.768      0.723      0.744      0.586



100 epochs completed in 0.354 hours.
Optimizer stripped from runs/detect/pit_2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/pit_2/weights/best.pt, 6.3MB

Validating runs/detect/pit_2/weights/best.pt...
Ultralytics YOLOv8.0.225 🚀 Python-3.10.12 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11167MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.95it/s]


                   all         97        252       0.81       0.69      0.749      0.591
                Type-1         97         39      0.962      0.657      0.761      0.554
                Type-2         97         69      0.699      0.696      0.703      0.531
                Type-3         97         62      0.818      0.653      0.722      0.616
                Type-4         97         82       0.76      0.756      0.809      0.663
Speed: 0.2ms preprocess, 3.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/pit_2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f3850aa2f80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0